In [4]:
import os
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import AUC
from tensorflow.keras import regularizers
import numpy as np
from sklearn.model_selection import train_test_split

train_dir = r'/home/toliman/PycharmProjects/PythonProgramming/deeplearning/DeepLearning/planes_data/avia-train'
test_dir = r'/home/toliman/PycharmProjects/PythonProgramming/deeplearning/DeepLearning/planes_data/avia-test'
train_labels_file = r'/home/toliman/PycharmProjects/PythonProgramming/deeplearning/DeepLearning/planes_data/train.csv'
n = 31080 * 4

file_names = os.listdir(train_dir)
file_names.sort(key=lambda x: int(x.split('.')[1]))

imsize_x, imsize_y = 32, 32

train_x = np.zeros(shape=(n, imsize_x, imsize_y, 3))
test_x = np.zeros(shape=(101000, imsize_x, imsize_y, 3))
train_y = list(map(lambda x: int(x[:-1]), open(train_labels_file).readlines()))
__t_y = np.zeros(shape=(n,))
for i, x in enumerate(train_y):
    __t_y[4 * i:4 * (i + 1)] = np.array([x, x, x, x])
train_y = __t_y

In [1]:
import cv2

for i, img in enumerate(file_names):
    # break
    image = Image.open(os.path.join(train_dir, img))
    img_arr = cv2.resize(np.asarray(image), (imsize_x, imsize_y), interpolation=cv2.INTER_CUBIC)
    try:
        train_x[i * 4] = img_arr
        train_x[i * 4 + 1] = np.rot90(img_arr, 1)
        train_x[i * 4 + 2] = np.rot90(img_arr, 2)
        train_x[i * 4 + 3] = np.rot90(img_arr, 3)
    except Exception:
        new_arr = np.zeros((imsize_x, imsize_y, 3))
        for x in range(imsize_x):
            for y in range(imsize_y):
                new_arr[x][y] = img_arr[x][y][:3]
        train_x[i * 4] = new_arr
        train_x[i * 4 + 1] = np.rot90(new_arr, 1)
        train_x[i * 4 + 2] = np.rot90(new_arr, 2)
        train_x[i * 4 + 3] = np.rot90(new_arr, 3)
    if i % 1000 == 0:
        print(f'{i // 1000}k')


NameError: name 'file_names' is not defined

In [3]:
d = []

# for i, filename in enumerate(os.listdir(test_dir)):
#     if i%1000 == 0:
#         print(f'renamed {i//1000}k')
#     d.append(filename)
#     os.rename(os.path.join(test_dir, filename), os.path.join(test_dir, f'img.{i}.png'))

test_file_names = os.listdir(test_dir)
test_file_names.sort(key=lambda x: int(x.split('.')[1]))

for i, img in enumerate(test_file_names):
    # break
    image = Image.open(os.path.join(test_dir, img))
    img_arr = cv2.resize(np.asarray(image), (imsize_x, imsize_y), interpolation=cv2.INTER_CUBIC)
    try:
        test_x[i] = img_arr
    except Exception:
        new_arr = np.zeros((imsize_x, imsize_y, 3))
        for x in range(imsize_x):
            for y in range(imsize_y):
                new_arr[x][y] = img_arr[x][y][:3]
        test_x[i] = new_arr
    if i % 1000 == 0:
        print(f'{i // 1000}k')

In [4]:
# test_seq = open(r'C:\Users\User\PycharmProjects\DeepLearning\planes_data\test_seq.txt', 'w')
# for x in d:
#     test_seq.write(x.split('.')[0]+'\n')
# test_seq.close()

In [5]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=.25)

In [3]:
from tensorflow.keras.applications import VGG16

netw = VGG16(weights='imagenet',
             include_top=False,
             input_shape=(imsize_x, imsize_y, 3))

netw.trainable = False

model = models.Sequential(
    [
        netw,
        layers.Flatten(),
        layers.Dense(192, activation=activations.elu, ),
        layers.Dense(1, activation='sigmoid'),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 19, 19, 32)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         32896     
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(learning_rate=.0002),
    metrics=[AUC(curve='ROC'), ]
)

In [ ]:
ep = 35

In [ ]:
history = model.fit(
    train_x,
    train_y,
    steps_per_epoch=50,
    epochs=ep,
    class_weight={0: 1, 1: 5},
    validation_data=(val_x, val_y),
    validation_steps=25,
)

model.save('planesnet.h5')

In [1]:
ans = model.predict(test_x)
s = 'sign,filename\n'

names = open(r'/home/toliman/PycharmProjects/PythonProgramming/deeplearning/DeepLearning/planes_data/test_seq.txt',
             'r').readlines()

for i, x in enumerate(ans):
    s += f'{1 if x[0] > .5 else 0},{names[i][:-1]}\n'

sub = open(r'/home/toliman/PycharmProjects/PythonProgramming/deeplearning/DeepLearning/planes_data/sub.csv',
           'w')
sub.write(s)
sub.close()

NameError: name 'network' is not defined

In [ ]:
print(history.history.keys())

auc, val_auc = history.history['auc'], history.history['val_auc']

plt.figure(figsize=(10, 10))
plt.plot(list(range(ep)), auc, color='blue')
plt.plot(list(range(ep)), val_auc, color='red')
plt.show()